In [2]:
import chromadb
from sentence_transformers import SentenceTransformer
import torch
from collections import defaultdict
import math

# ---------------------------------------------------------
# 1. 설정 (GPU 확인 등)
# ---------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ---------------------------------------------------------
# 2. 모델 로드 (저장할 때 썼던 그 모델!)
# ---------------------------------------------------------
print("모델 로드 중...")
model = SentenceTransformer("dragonkue/BGE-m3-ko").to(device)

# ---------------------------------------------------------
# 3. ChromaDB 연결 (다운로드 받은 폴더 경로 지정)
# ---------------------------------------------------------
# path="./chroma_db" 는 압축 푼 폴더 이름과 같아야 합니다.
client = chromadb.PersistentClient(path="../../../db_search/doc_db")

# 컬렉션 가져오기 (create가 아니라 get_collection 사용)
collection = client.get_collection(name="patent_claims")

print(f"✅ 데이터베이스 로드 완료! 총 데이터 수: {collection.count()}개")



c:\Users\kouls\anaconda3\envs\llm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
모델 로드 중...
✅ 데이터베이스 로드 완료! 총 데이터 수: 589049개


In [3]:
# 200개 초과 검색되었을 때 re-ranking (z 정규화 수정ver)
import numpy as np

def multi_query_rerank(
    collection,
    model,
    query_list,
    per_query_top_k=200,
    final_top_k=200
):
    #------------------------------------------
    # 1) Q개의 query 문장 embedding
    #------------------------------------------
    query_embs = model.encode(query_list).tolist()


    #------------------------------------------
    # 2-1) query_list가 한 개일 경우, 검색 후 바로 return
    #------------------------------------------
    if len(query_list) == 1:
        return collection.query(query_embeddings=query_embs, n_results=final_top_k)


    #------------------------------------------
    # 2-2) query별 검색
    #------------------------------------------
    candidates = []  # 전체 후보 저장
    for emb in query_embs:
        r = collection.query(
            query_embeddings=[emb],
            n_results=per_query_top_k
        )
    
        distances = np.array(r["distances"][0])
        mean = distances.mean()
        std = distances.std() + 1e-9

        z_scores = (distances - mean) / std

        ids = r["ids"][0]
        docs = r["documents"][0]
        distances = r["distances"][0]
        metas = r["metadatas"][0]

        # 후보를 통합 리스트에 추가
        for pid, doc, meta, z, dist in zip(ids, docs, metas, z_scores, distances):
            candidates.append({
                "id": pid,
                "document": doc,
                'metadatas':meta,
                'distance':dist,
                'z-score':z
            })

    #------------------------------------------
    # 3) z-score 기준 오름차순 정렬 후 상위 final_top_k만 선택
    #------------------------------------------
    top_candidates = sorted(candidates, key=lambda x: x["z-score"])[:final_top_k]


    #------------------------------------------
    # 4) collection.query() 형식으로 재구성
    #------------------------------------------
    final_ids = [c["id"] for c in top_candidates]
    final_docs = [c["document"] for c in top_candidates]
    final_distances = [c["distance"] for c in top_candidates]
    final_metas = [c['metadatas'] for c in top_candidates]

    final_results = {
        "ids": [final_ids],
        "documents": [final_docs],
        "distances": [final_distances],
        "metadatas": [final_metas]
    }

    return final_results

In [4]:
import numpy as np

def many_claim_dis(results, TOP_K):
    # ----------------------------------------
    # 0. Chroma 결과 파싱
    # ----------------------------------------
    ids        = results["ids"][0]
    docs       = results["documents"][0]
    metas      = results["metadatas"][0]
    distances  = results["distances"][0]

    parsed = []
    for i in range(len(ids)):
        parsed.append({
            "id": ids[i],
            "document": docs[i],
            "metadata": metas[i],
            "distance": distances[i]
        })

    # ----------------------------------------
    # 1. 출원번호 기준 그룹화
    # ----------------------------------------
    grouped = defaultdict(list)
    for r in parsed:
        app_no = r["metadata"]["patent_id"]
        grouped[app_no].append(r)

    # ----------------------------------------
    # 2. 특허 단위 점수 계산
    #    방법: claim similarity들의 평균 + 대표 claim 보정
    # ----------------------------------------

    def similarity(d):
        return 1-d


    def compute_patent_score(claims):
        sims = [similarity(c["distance"]) for c in claims]   # 새로운 sim

        sims_sorted = sorted(sims, reverse=True)
        top3 = sims_sorted[:3]
        top3_avg = sum(top3) / len(top3)
        max_sim = sims_sorted[0]

        claim_count = len(claims)
        count_bonus = min(1.0, claim_count / 10.0)

        final_score =  top3_avg * 0.6 + max_sim * 0.3 + count_bonus * 0.1
    
        return final_score

    # ----------------------------------------
    # 3. 특허 단위 재랭킹
    # ----------------------------------------
    aggregated = []
    for app_no, claims in grouped.items():
        score = compute_patent_score(claims)

        # 대표 claim은 거리(distance)가 가장 낮은 claim 선택
        rep_claim = sorted(claims, key=lambda x: x["distance"])[0]
         # claims에서 id와 metadata를 제외하고 document와 distance만 저장
        
        filtered_claims = [
            {
                "id": c["id"],
                "document": c["document"],
                "distance": c["distance"]
            }
            for c in claims
        ]

        aggregated.append({
            "patent_id": app_no,
            "score": score,
            "top_claim": rep_claim["document"],
            "top_claim_no": rep_claim["metadata"]["claim_no"],
            "claims_found": len(claims),
            "claims": filtered_claims
        })

    # 점수 높은 순으로 재랭킹
    aggregated = sorted(aggregated, key=lambda x: x["score"], reverse=True)

    final_response = aggregated[:TOP_K]
    return final_response

In [5]:
query = ["인공지능 기반 패션 트렌드 분석 및 의류 제조 시스템"]

# multi_query_rerank 실행 (이 부분은 유지)
results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

# --- 수정된 데이터 추출 및 중복 제거 로직 ---

# results에서 필요한 데이터 추출
ids = results["ids"][0]
docs = results["documents"][0]
metadatas = results["metadatas"][0] # <-- patent_id가 포함된 메타데이터

seen_patents = set()
top_30_patents = []

# 순서를 유지하면서 중복 제거
for i in range(len(ids)):
    # patent_id는 metadatas에서 가져와야 합니다.
    patent_id = metadatas[i].get("patent_id") 
    
    if patent_id and patent_id not in seen_patents: # patent_id가 존재하는지 확인
        seen_patents.add(patent_id)
        
        # claim은 docs에서 가져옵니다.
        claim = docs[i]
        
        top_30_patents.append({
            "patent_id": patent_id,
            "claim": claim
        })
        
        # 30개 모으면 중단
        if len(top_30_patents) >= 30:
            break

print(top_30_patents)

[{'patent_id': '1020240180816', 'claim': '인공지능 기반으로 최신 트렌드에 부합하는 의류를 생산하기 위한 시스템에 있어서,인공지능에 기반하여 신규 의류 디자인을 창출하는 서버를 포함하고,상기 서버는, 데이터 수집부; 데이터 분석부; 의류 디자인 관리부; 및 데이터베이스를 포함하고,상기 데이터 수집부는,SNS(Social Network Service)에 업로드된 게시물 중 의류가 포함된 복수의 이미지를 주기적으로 수집하고,상기 데이터 분석부는,상기 수집된 복수의 이미지 각각에서 의류 이미지에 해당하는 영역을 분리하고,상기 분리된 의류 이미지를 의류 디자인의 유행을 예측하도록 학습된 인공지능 모델에 입력하고, 상기 인공지능 모델을 이용하여, 상기 의류 이미지의 카테고리를 분류하여 상기 카테고리 별로 의류 이미지의 특징을 추출하고,상기 인공지능 모델을 이용하여, 상기 추출된 특징에 기초하여 상기 의류 이미지를 브랜드명, 스타일, 색상, 원단의 재질 및 패턴을 포함하는 속성으로 분류하고,상기 분류된 속성에 기초하여 유행할 의류 디자인에 대한 예측 결과를 출력하고,상기 의류 디자인 관리부는,상기 예측 결과에 기초하여 생성된 신규 의류 디자인에 따른 의류의 생산을 의뢰하기 위한 문서를 출력하고,상기 의류 디자인 관리부는,상기 의류 이미지에 대응하는 의류 또는 상기 신규 의류 디자인에 따른 의류의 제1 판매 기간이 도과한 후, 상기 의류에 대한 소비자의 리뷰 정보를 획득하고,상기 리뷰 정보에 기초하여, 상기 의류의 스타일, 색상, 원단의 재질 및 패턴을 포함하는 속성에 대한 평가 점수를 출력하고,상기 데이터베이스에서 기준값 이상의 평가 점수를 나타내는 의류에 대응하는 생산 업체를 확인하고,상기 기준값 이상의 평가 점수를 나타내는 의류에 기초하여 생성된 신규 의류 디자인을 수신하고,상기 생산 업체의 공정 일정을 미리 확인하여 생산 의뢰 일정을 결정하고,제2 판매 기간 동안의 판매를 위해, 상기 생산 업체에 상기 신규 의류 디자인에 따른 의류의

In [6]:
top_claims_list = [item['claim'] for item in top_30_patents]
top_claims_list

['인공지능 기반으로 최신 트렌드에 부합하는 의류를 생산하기 위한 시스템에 있어서,인공지능에 기반하여 신규 의류 디자인을 창출하는 서버를 포함하고,상기 서버는, 데이터 수집부; 데이터 분석부; 의류 디자인 관리부; 및 데이터베이스를 포함하고,상기 데이터 수집부는,SNS(Social Network Service)에 업로드된 게시물 중 의류가 포함된 복수의 이미지를 주기적으로 수집하고,상기 데이터 분석부는,상기 수집된 복수의 이미지 각각에서 의류 이미지에 해당하는 영역을 분리하고,상기 분리된 의류 이미지를 의류 디자인의 유행을 예측하도록 학습된 인공지능 모델에 입력하고, 상기 인공지능 모델을 이용하여, 상기 의류 이미지의 카테고리를 분류하여 상기 카테고리 별로 의류 이미지의 특징을 추출하고,상기 인공지능 모델을 이용하여, 상기 추출된 특징에 기초하여 상기 의류 이미지를 브랜드명, 스타일, 색상, 원단의 재질 및 패턴을 포함하는 속성으로 분류하고,상기 분류된 속성에 기초하여 유행할 의류 디자인에 대한 예측 결과를 출력하고,상기 의류 디자인 관리부는,상기 예측 결과에 기초하여 생성된 신규 의류 디자인에 따른 의류의 생산을 의뢰하기 위한 문서를 출력하고,상기 의류 디자인 관리부는,상기 의류 이미지에 대응하는 의류 또는 상기 신규 의류 디자인에 따른 의류의 제1 판매 기간이 도과한 후, 상기 의류에 대한 소비자의 리뷰 정보를 획득하고,상기 리뷰 정보에 기초하여, 상기 의류의 스타일, 색상, 원단의 재질 및 패턴을 포함하는 속성에 대한 평가 점수를 출력하고,상기 데이터베이스에서 기준값 이상의 평가 점수를 나타내는 의류에 대응하는 생산 업체를 확인하고,상기 기준값 이상의 평가 점수를 나타내는 의류에 기초하여 생성된 신규 의류 디자인을 수신하고,상기 생산 업체의 공정 일정을 미리 확인하여 생산 의뢰 일정을 결정하고,제2 판매 기간 동안의 판매를 위해, 상기 생산 업체에 상기 신규 의류 디자인에 따른 의류의 생산을 의뢰하기 위한 문서를 출력하고, 상기 문서는 상기 생산 의뢰 일

In [7]:
query = ["인공지능 기반 패션 트렌드 분석 및 의류 제조 시스템"]

results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

results = many_claim_dis(results, TOP_K=30)

In [8]:
top_claims_list = [item['top_claim'] for item in results]
top_claims_list

['인공지능 기반으로 최신 트렌드에 부합하는 의류를 생산하기 위한 시스템에 있어서,인공지능에 기반하여 신규 의류 디자인을 창출하는 서버를 포함하고,상기 서버는, 데이터 수집부; 데이터 분석부; 의류 디자인 관리부; 및 데이터베이스를 포함하고,상기 데이터 수집부는,SNS(Social Network Service)에 업로드된 게시물 중 의류가 포함된 복수의 이미지를 주기적으로 수집하고,상기 데이터 분석부는,상기 수집된 복수의 이미지 각각에서 의류 이미지에 해당하는 영역을 분리하고,상기 분리된 의류 이미지를 의류 디자인의 유행을 예측하도록 학습된 인공지능 모델에 입력하고, 상기 인공지능 모델을 이용하여, 상기 의류 이미지의 카테고리를 분류하여 상기 카테고리 별로 의류 이미지의 특징을 추출하고,상기 인공지능 모델을 이용하여, 상기 추출된 특징에 기초하여 상기 의류 이미지를 브랜드명, 스타일, 색상, 원단의 재질 및 패턴을 포함하는 속성으로 분류하고,상기 분류된 속성에 기초하여 유행할 의류 디자인에 대한 예측 결과를 출력하고,상기 의류 디자인 관리부는,상기 예측 결과에 기초하여 생성된 신규 의류 디자인에 따른 의류의 생산을 의뢰하기 위한 문서를 출력하고,상기 의류 디자인 관리부는,상기 의류 이미지에 대응하는 의류 또는 상기 신규 의류 디자인에 따른 의류의 제1 판매 기간이 도과한 후, 상기 의류에 대한 소비자의 리뷰 정보를 획득하고,상기 리뷰 정보에 기초하여, 상기 의류의 스타일, 색상, 원단의 재질 및 패턴을 포함하는 속성에 대한 평가 점수를 출력하고,상기 데이터베이스에서 기준값 이상의 평가 점수를 나타내는 의류에 대응하는 생산 업체를 확인하고,상기 기준값 이상의 평가 점수를 나타내는 의류에 기초하여 생성된 신규 의류 디자인을 수신하고,상기 생산 업체의 공정 일정을 미리 확인하여 생산 의뢰 일정을 결정하고,제2 판매 기간 동안의 판매를 위해, 상기 생산 업체에 상기 신규 의류 디자인에 따른 의류의 생산을 의뢰하기 위한 문서를 출력하고, 상기 문서는 상기 생산 의뢰 일

In [10]:
from hybrid_search_function import hybrid_search
# 1. 검색 실행
query = ["인공지능 기반 패션 트렌드 분석 및 의류 제조 시스템"]

search_results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

patent_results = hybrid_search(
    multi_query_results=search_results,
    query_list=query,
    top_k=30,
    vector_weight=0.7,
    bm25_weight=0.3
)

top_claims_list = [item['top_claim'] for item in patent_results]
top_claims_list

['상기 시스템 서버는,상기 음성 추출부를 통해 추출된 텍스트를 기반으로 카테고리 정보를 생성하고, 여기서 생성된 카테고리 정보를 학습하여 트랜드 정보를 생성하는 인공지능 분석부;를 더 포함하여 구성되는 동영상 컨텐츠 플랫폼에서 범죄 악용 컨텐츠 패턴 분석 시스템.',
 '상기 통합 서비스 제공 서버는,상기 AI 렌즈를 구축하기 위하여 패션상품의 종류, 무늬, 색상 및 용도를 포함하는 분석 데이터와, 적어도 하나의 사진을 데이터셋(DataSet)으로 구축하고, 적어도 하나의 인공지능 알고리즘에 패션상품의 사진을 질의(Query)로 입력하면, 상기 패션상품의 종류, 무늬, 색상 및 용도를 출력하도록 학습 및 검증한 후, 정확도가 가장 높은 인공지능 알고리즘을 상기 AI 렌즈로 세팅하는 모델링부;를 더 포함하는 것을 특징으로 하는 인공지능 기반 패션 통합 서비스 제공 시스템.',
 '인공지능 기반의 영상 분석을 통한 부품 결함 검사 시스템에 있어서,대상 부품의 표면 영역을 포함하는 분석 대상 이미지를 촬영하는 촬영 모듈;상기 분석 대상 이미지를 수신하고, 복수의 부품 각각의 표면 영역을 촬영한 학습 이미지 및 상기 학습 이미지 상에 등장하는 결함 영역에 대한 라벨 정보를 포함하는 학습 데이터에 기초한 학습을 통해 생성된 인공지능 기반의 분석 모델을 이용하여 상기 분석 대상 이미지에 반영된 상기 대상 부품의 표면 특성에 따른 상기 대상 부품에 대한 결함 존부에 대한 정보 및 결함 유형에 대한 정보를 포함하는 결함 정보를 출력하는 분석 모듈; 및상기 분석 대상 이미지 및 상기 결함 정보를 표시하는 디스플레이 모듈,을 포함하는, 검사 시스템.',
 '패션 이미지 분석 기반의 대시보드 서비스 제공 방법에 있어서,복수의 패션 이미지를 수집하고, 상기 복수의 패션 이미지 각각의 미리 설정된 범주 별 식별 정보를 획득하는 단계;기 학습된 인공지능 기반의 제1특성 추출 모델을 이용하여 상기 복수의 패션 이미지 각각에 반영된 패션 요소를 추출하는 단계;사용자 단말로부터 탐색하고자